In [1]:
import torch
from torch import nn
from torchvision import models as CVModels

In [2]:
resnet18 = CVModels.resnet18(pretrained=True)

/Users/tanknee/miniforge3/envs/tsr/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tanknee/miniforge3/envs/tsr/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
backbone = nn.Sequential(
    resnet18.conv1,
    resnet18.bn1,
    resnet18.relu,
    resnet18.maxpool,
    resnet18.layer1,
    nn.Conv2d(64, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU()
)


In [4]:
appearance = torch.randn(8, 3, 512, 512)

In [13]:
cnn_feat = backbone(appearance)

In [14]:
cnn_feat.shape

torch.Size([8, 64, 128, 128])

In [32]:
bounding_box = torch.randn(10, 4)
boxes = []
for i in range(8):
    boxes.append(bounding_box)

In [33]:
from torchvision.ops import roi_align

# bounding_box = torch.randn(10, 4)
# # 重复8次
# bounding_box = bounding_box.repeat(8, 1)


align_feat = roi_align(cnn_feat, boxes, 2)

In [35]:
cnn_feat.shape

torch.Size([8, 64, 128, 128])

In [37]:
align_feat.shape

torch.Size([80, 64, 2, 2])

In [38]:
fc=nn.Sequential(
    nn.Linear(2 ** 2 * 64, 64),
    nn.ReLU(),
)

In [45]:
# get shape of align_feat
fc(align_feat.view(align_feat.shape[0], -1)).view(cnn_feat.shape[0], -1, 64).shape

torch.Size([8, 10, 64])